# Aspects (plus) avancés pour la bioinformatique: exécution d'un programme distant via un URL

## Comment faire une recherche BLAST en utilisant l'application hébergée au NCBI

Nous n'avons pas nécessairement toutes les applications voulues sur notre poste de travail ou sur notre serveur... Il est possible d'exécuter une application à distance en tant que *web service*, c'est-à-dire accessible via un URL.

Le service [BLAST URL API](https://blast.ncbi.nlm.nih.gov/doc/blast-help/developerinfo.html#developerinfo) permet de faire des alignements BLAST sur les serveurs du NCBI avec un simple (en fait pas si simple...) URL... mais pas directement :-(

La tâche se déroule en trois temps:

  - Il faut soumettre la tâche avec l'URL de soumission en spécifiant la séquence, le programme BLAST à utiliser, la base de données et tout autre paramètre pour faire le travail;
  - Apràs la soumission, le serveur BLAST nous retourne un flux de données qui est en fait une page HTML. Dans tout ce code, il n'y a qu'un seul élément important pour nous, c'est le RID, c'est-à-dire l'identificateur de notre requête. Il est nécessaire pour récupérer le résultat de la tâche car cette méthode n'est pas interactive...
  - Pour terminer, il nous faut récupérer nos résultats grâce à un autre URL permettant de limiter le nombre de résultats ou bien le format de sortie du résultat (texte ou bien XML par exemple).


#### Pré-requis nécessaire

Il nous faut installer une librairie appelée *BeautifulSoup* qui nous permettra d'extraire la valeur du champ RID: 

In [10]:
!pip install beautifulsoup4

#### Code de démonstration

In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep

# Demandé par le NCBI pour faire le suivi des requetes
email = "sylvain.foisy@umontreal.ca"
tool = "bcm3553_demo_blast"

#
# Consultez la documentation NCBI pour modifier selon vos besoins
#
# La liste des paramétres BLAST utilisables est ici:
# https://blast.ncbi.nlm.nih.gov/doc/blast-help/urlapi.html#urlapi
#
# Parametre "PROGRAM"
blastTool = "blastn"
# Parametre "DATABASE"
blastDb = "core_nt"

#
# Lire un fichier FASTA, ici de nucléotides
# On se casse pas la tête avec la vérification...
#
readFile = open("../z.misc_files/data_seq/h.sapiens.cttn.iso.a.cdna.fa","r")
seq = ""
allLines = readFile.readlines()
#
# Quelle est la logique à utiliser? L'URL ne peut contenir que des caractères
# conformes à l'alphabet IUPAC; il nous fait donc retirer le '>'. De plus,
# l'URL ne peut contenir de retour de chariot dans la sequence envoyée; on
# doit donc concaténer chaque ligne pour n'en faire qu'une seule
#
for aLine in allLines:
    if aLine[0]!='>':
        aLine  = aLine.replace('\n','')
        seq  = seq+aLine

#
# Portion statique de l'URL
#
blast_url = "https://blast.ncbi.nlm.nih.gov/Blast.cgi?"

#
# Portion spécifique à la soumission de la requete
#
# On utilise la méthode format pour envoyer les champs sequentiellement 
# dans la chaine de caracteres
#
cmds = "CMD=Put&email={}&tool={}&PROGRAM={}&DATABASE={}&QUERY={}".format(email,tool,blastTool,blastDb,seq)
blast_url_post = blast_url+cmds

# Preparation de la requete
headers = {'content-type': 'application/x-www-form-urlencoded','connection':'keep-alive'}

# Envoyez la requete
res = requests.get(blast_url_post,headers = headers)
#
# Creons un objet BeautifulSoup contenant les donnes retournees
#
bowl = BeautifulSoup(res.text,"html")
#
# Recherchons LA ligne qui nous intéresse et
# extrayons la donnee recherchee
#
# Logique: dans toutes les lignes avec le tag input, il nous faut trouver
#          celle qui contient le champs rid
#
ridLine = bowl.find("input", attrs={'id':"rid"})
rid = ridLine.get('value')

#
# Une fois que l'analyse est complétée, utiliser le RID pour récupérer vos résultats d'alignement
#
# Pour laisser le temps à l'analyse de se faire et ne pas submerger le serveur...
sleep(120)

#
# Portion specifique a la recuperation des resultats
#
cmds = "CMD=Get&FORMAT_TYPE=Text&RID={}".format(rid)
blast_url_get = blast_url+cmds

out = requests.get(blast_url_get,headers=headers)

#
# Écrivons nos résultats dans un fichier HTML
#
writeFile = open("../z.misc_files/data_seq/h.sapiens.cttn.iso.a.cdna.blastn.txt","w")
writeFile.write(out.text)
writeFile.close()